In [205]:
import pandas as pd 
import numpy as np 
import infrastructure
import filepath 
import pygsheets
from datetime import datetime, timedelta
import openpyxl 
import os
from pathlib import Path
import xlsxwriter
import send_email

pd.set_option('display.max_columns', None)


In [206]:
#Publisher Config
gc = pygsheets.authorize(service_account_file=filepath.service_account_location)
publisher_config = gc.open_by_url(
    'https://docs.google.com/spreadsheets/d/1Tzda6Djr3zQmOhWu7Ief3GVR9Cjaml8238CeX7chj_U/edit#gid=1620368362') 
pub_config = publisher_config[0].get_as_df()
pub_config.rename(columns={'DP.DS or DP.sV': 'DP.SV'}, inplace=True)

In [207]:
#AR Smartsheet
sms_AR = infrastructure.get_smartsheet('ar_sms')


In [208]:
sms_AR['Offer ID'] = sms_AR['New Offer Name'].str.split('_').str[-1]


In [209]:
#manual input
df = pd.read_csv('SS_LC_merged_data.csv')

/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,4,6,8,9,10,11,12,14,16,26,27,28,29,30,31,32,33,34,35,36,37,44,48,51,54,56,58,59,63) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [210]:
# Filter the DataFrame by Send Strategy
ar_df = df[df['Send Strategy'] == 'AR']


In [211]:
ar_df['shortcode_DP.SV'] = ar_df['shortcode_DP.SV'].apply(lambda x: str(x)[:min(3, str(x).index('_'))] + str(x)[str(x).index('_'):] if '_' in str(x) else str(x))

<ipython-input-211-2f2f4f8aaa10>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar_df['shortcode_DP.SV'] = ar_df['shortcode_DP.SV'].apply(lambda x: str(x)[:min(3, str(x).index('_'))] + str(x)[str(x).index('_'):] if '_' in str(x) else str(x))


In [212]:
# remove only the ".0" at the end of the Hitpath_Offer_ID column in ar_df
ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')


<ipython-input-212-c0d54c8234f7>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')
<ipython-input-212-c0d54c8234f7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ar_df['Hitpath_Offer_ID'] = ar_df['Hitpath_Offer_ID'].str.replace(r'\.0$', '')


In [213]:
#get inactive pubid from pub_config
inactive_pubid = pub_config[pub_config['INTERNAL STATUS'] == 'INACTIVE']['PUBID'].tolist()

In [214]:
# convert inactitve_pubid to float type
inactive_pubid = [float(i) for i in inactive_pubid if i != '']

In [215]:
#remove inactive pubid from ar_df
ar_df = ar_df[~ar_df['Affiliate_Id'].isin(inactive_pubid)]

Last 30 day data

In [216]:

# Convert 'Date' column to datetime type
ar_df['Date'] = pd.to_datetime(ar_df['Date'])

# Calculate the date 37 days ago
thirty_days_ago = datetime.now() - timedelta(days=37)

# Calculate the date 7 days ago
seven_days_ago = datetime.now() - timedelta(days=7)

# Filter the DataFrame for the last 30 days after the last 7 days
ar30_df = ar_df[(ar_df['Date'] <= seven_days_ago) & (ar_df['Date'] >= thirty_days_ago)]

# Filter the DataFrame for the last 7 days
ar7_df = ar_df[ar_df['Date'] >= seven_days_ago]

In [217]:
#filter 30 day df
filtered_ar30_df = ar30_df[['Hitpath_Offer_ID', 'Affiliate_Id', 'DP&Pub', 'Revenue', 
                            'Jump Page Clicks', 'Delivered', 'Optout', 'Clicks', 'Cost', 
                            'Ar Day', 'Shortcode Name', 'DP.SV', 'shortcode_DP.SV', 
                            'Opportunity Cost', 'Offer Vertical']]

#filter 7 day df
filtered_ar7_df = ar7_df[['Hitpath_Offer_ID', 'Affiliate_Id', 'DP&Pub', 'Revenue',
                            'Jump Page Clicks', 'Delivered', 'Optout', 'Clicks', 'Cost',
                            'Ar Day', 'Shortcode Name', 'DP.SV', 'shortcode_DP.SV',
                            'Opportunity Cost', 'Offer Vertical']]

In [218]:
grouped_ar30_df = filtered_ar30_df.groupby(['shortcode_DP.SV', 'Ar Day']).agg({'Revenue': 'sum', 'Cost': 'sum', 'Delivered': 'sum', 'Jump Page Clicks': 'sum', 'Clicks': 'sum', 'Opportunity Cost': 'sum', 'Optout': 'sum'}).reset_index()
grouped_ar7_df = filtered_ar7_df.groupby(['shortcode_DP.SV', 'Ar Day']).agg({'Revenue': 'sum', 'Cost': 'sum', 'Delivered': 'sum', 'Jump Page Clicks': 'sum', 'Clicks': 'sum', 'Opportunity Cost': 'sum', 'Optout': 'sum'}).reset_index()

In [219]:
#30 day averages
grouped_ar30_df['30 Day eCPM'] = grouped_ar30_df['Revenue'] / grouped_ar30_df['Delivered'] * 1000
grouped_ar30_df['30 Day CTR'] = grouped_ar30_df['Clicks'] / grouped_ar30_df['Delivered']
grouped_ar30_df['30 Day JCTR'] = grouped_ar30_df['Jump Page Clicks'] / grouped_ar30_df['Delivered']
grouped_ar30_df['30 Day Optout Rate'] = grouped_ar30_df['Optout'] / grouped_ar30_df['Delivered']

#7 day averages
grouped_ar7_df['7 Day eCPM'] = grouped_ar7_df['Revenue'] / grouped_ar7_df['Delivered'] * 1000
grouped_ar7_df['7 Day CTR'] = grouped_ar7_df['Clicks'] / grouped_ar7_df['Delivered']
grouped_ar7_df['7 Day JCTR'] = grouped_ar7_df['Jump Page Clicks'] / grouped_ar7_df['Delivered']
grouped_ar7_df['7 Day Optout Rate'] = grouped_ar7_df['Optout'] / grouped_ar7_df['Delivered']

In [220]:
# #percentages
grouped_ar30_df['30 Day CTR'] = grouped_ar30_df['30 Day CTR'] * 100
grouped_ar30_df['30 Day JCTR'] = grouped_ar30_df['30 Day JCTR'] * 100
grouped_ar30_df['30 Day Optout Rate'] = grouped_ar30_df['30 Day Optout Rate'] * 100

grouped_ar7_df['7 Day CTR'] = grouped_ar7_df['7 Day CTR'] * 100
grouped_ar7_df['7 Day JCTR'] = grouped_ar7_df['7 Day JCTR'] * 100
grouped_ar7_df['7 Day Optout Rate'] = grouped_ar7_df['7 Day Optout Rate'] * 100 

In [221]:
merged_df = grouped_ar30_df.merge(grouped_ar7_df[['shortcode_DP.SV','Ar Day', '7 Day eCPM', '7 Day Optout Rate']], 
                                 on=['shortcode_DP.SV', 'Ar Day'], 
                                 how='left')

In [222]:
# calculate percent difference between 30 day and 7 day eCPM with 30 day eCPM as the base
merged_df['eCPM % Diff'] = (merged_df['7 Day eCPM'] - merged_df['30 Day eCPM']) / merged_df['30 Day eCPM'] * 100
merged_df['Optout Rate % Diff'] = merged_df['7 Day Optout Rate'] - merged_df['30 Day Optout Rate']

Most recent Offer ID for AR and Active

In [223]:
#Get Active SMS ARs from sms_ar
active_sms_ar = sms_AR[sms_AR['Status'] == 'Active']
active_sms_ar['shortcode_DP.SV'] = active_sms_ar['Shortcode'] + '_' + active_sms_ar['DP.DS']
#remove AR portion from Day column and convert the remaining number to an integer
active_sms_ar['Day'] = active_sms_ar['Day'].str.replace('AR', '').astype(float)
active_sms_ar.rename(columns={'Day': 'Ar Day'}, inplace=True)


<ipython-input-223-ce9b7e1a9952>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_sms_ar['shortcode_DP.SV'] = active_sms_ar['Shortcode'] + '_' + active_sms_ar['DP.DS']
<ipython-input-223-ce9b7e1a9952>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_sms_ar['Day'] = active_sms_ar['Day'].str.replace('AR', '').astype(float)
/Users/nathan/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame



In [224]:
#get active ARs from merged df
merged_df = merged_df.merge(active_sms_ar[['shortcode_DP.SV', 'Ar Day']], on=['shortcode_DP.SV', 'Ar Day'], how='inner')

In [225]:
merged_df = merged_df.merge(active_sms_ar[['shortcode_DP.SV', 'Ar Day', 'Offer ID', 'Date Started']], on=['shortcode_DP.SV', 'Ar Day'], how='left')
#convert Date Started to datetime
merged_df['Date Started'] = pd.to_datetime(merged_df['Date Started'])

#days since swap
merged_df['Days Since Swap'] = (datetime.now() - timedelta(days=1) - merged_df['Date Started']).dt.days

In [226]:
cols = list(merged_df.columns)
cols.insert(2, cols.pop(cols.index('Offer ID')))
merged_df = merged_df[cols]

In [227]:
# #create a new column called AR Check, if eCPM % Diff is less than -25 and Days Since Swap is greater than 7, then AR Check is 'Underperforming', else if ecpm % diff is less than -25 and days since swap is less than 7, then AR Check is 'In Review', else 'Good'
# merged_df['AR Check'] = merged_df.apply(lambda row: 'Downtrending - Swap' if row['eCPM % Diff'] < -30 and row['Days Since Swap'] > 37 
#                                         else ('In Review' if row['eCPM % Diff'] < -30 and row['Days Since Swap'] <= 7
#                                         else ('Underperforming Swap' if row['eCPM % Diff'] < -30 and row['Days Since Swap'] > 7 and row['Days Since Swap'] <= 37 else 'Good')), 
#                                         axis=1)

#if AR Check is 'Good' but 30 Day eCPM is less than 7, then AR Check is 'Underperforming'
# merged_df['AR Check'] = merged_df.apply(lambda row: 'Underperforming' if row['AR Check'] == 'Good' and row['30 Day eCPM'] < 7 else row['AR Check'], axis=1)

In [228]:
merged_df['AR Check'] = merged_df.apply(lambda row: 
                                        'Downtrending - Swap' if ((row['30 Day eCPM'] > 14 and row['eCPM % Diff'] < -40 and row['Days Since Swap'] > 37) or 
                                                                  (row['30 Day eCPM'] >= 7 and row['30 Day eCPM'] <= 14 and row['eCPM % Diff'] < -25 and row['Days Since Swap'] > 37)) 
                                        else ('In Review - Swap' if ((row['30 Day eCPM'] > 14 and row['eCPM % Diff'] < -40) or 
                                                                     (row['30 Day eCPM'] >= 7 and row['30 Day eCPM'] <= 14 and row['eCPM % Diff'] < -25)) and row['Days Since Swap'] <= 7 
                                        else ('Underperforming Swap' if ((row['30 Day eCPM'] > 14 and row['eCPM % Diff'] < -40) or 
                                                                         (row['30 Day eCPM'] >= 7 and row['30 Day eCPM'] <= 14 and row['eCPM % Diff'] < -25)) and row['Days Since Swap'] > 7 and row['Days Since Swap'] <= 37 
                                        else 'Good')), axis=1)

merged_df['AR Check'] = merged_df.apply(lambda row: 'Below Cost eCPM' if row['AR Check'] == 'Good' and row['7 Day eCPM'] < 7 else row['AR Check'], axis=1)
merged_df['AR Check'] = merged_df.apply(lambda row: 'In Review - Swap' if row['AR Check'] == 'Below Cost eCPM' and row['7 Day eCPM'] < 7 and row['Days Since Swap'] <= 7 else row['AR Check'], axis=1)

merged_df['AR Check'] = merged_df.apply(lambda row: 'Optout Rate Increase' if row['AR Check'] == 'Underperforming Swap' and row['Optout Rate % Diff'] >= 2 else row['AR Check'], axis=1)

#check if 7 day eCPM exists, if not, then AR Check is "Potential Data Issue"
merged_df['AR Check'] = merged_df.apply(lambda row: 'Potential Data Issue' if pd.isnull(row['7 Day eCPM']) else row['AR Check'], axis=1)

In [229]:
#convert merged_df Offer ID to string
# merged_df['Offer ID'] = merged_df['Offer ID'].astype(str).str.replace(r'\.0$', '')

In [230]:
# BY DMA Account
merged_df['extracted_DP.SV'] = merged_df['shortcode_DP.SV'].str.split('_').str[-1]

#merge using 'extracted_DP.SV' and 'DP.SV'
merged_df = pd.merge(merged_df, pub_config[['DP.SV', 'DMA']], 
                     left_on='extracted_DP.SV', right_on='DP.SV', how='left')

#drop temp columns
merged_df = merged_df.drop(columns=['extracted_DP.SV', 'DP.SV'])

dma_separated_dfs = {}
for dma in merged_df['DMA'].unique():
    if pd.notna(dma):  # Ensure that the 'DMA' value is not NaN
        dma_separated_dfs[dma] = merged_df[merged_df['DMA'] == dma]

In [231]:
#new dataframes for statuses sorted
good = merged_df[merged_df['AR Check'] == 'Good']
good = good.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

downtrending_swaps = merged_df[merged_df['AR Check'] == 'Downtrending - Swap']
downtrending_swaps = downtrending_swaps.sort_values(by=['DMA', 'shortcode_DP.SV','Days Since Swap'], ascending=[True, True, False])

in_review = merged_df[merged_df['AR Check'] == 'In Review - Swap']
in_review = in_review.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

underperforming_swaps = merged_df[merged_df['AR Check'] == 'Underperforming Swap']
underperforming_swaps = underperforming_swaps.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

below_cost = merged_df[merged_df['AR Check'] == 'Below Cost eCPM']
below_cost = below_cost.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

optout_rate_increase = merged_df[merged_df['AR Check'] == 'Optout Rate Increase']
optout_rate_increase = optout_rate_increase.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])

potential_data_issue = merged_df[merged_df['AR Check'] == 'Potential Data Issue']
potential_data_issue = potential_data_issue.sort_values(by=['DMA','shortcode_DP.SV', 'Days Since Swap'], ascending=[True, True, False])


In [232]:
#output file
filename = filepath.output_folder+'/AR Offer Alert - {}.xlsx'.format(datetime.today().strftime("%m_%d_%Y"))

#create writer
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
with pd.ExcelWriter(filename) as writer:
    for dma, df in dma_separated_dfs.items():
        sheet_name = f'{dma} - AR Offers'  # dynamic sheet name based on the DMA
        df.to_excel(writer, sheet_name=sheet_name, index=False)  # Writing data to different sheets

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    good.to_excel(writer, sheet_name='Good', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    downtrending_swaps.to_excel(writer, sheet_name='Downtrending - Swap', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    underperforming_swaps.to_excel(writer, sheet_name='Underperforming Swaps', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    in_review.to_excel(writer, sheet_name='In Review - Swap', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    below_cost.to_excel(writer, sheet_name='Below Cost', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    optout_rate_increase.to_excel(writer, sheet_name='Optout Rate Increase', index=False)

with pd.ExcelWriter(filename, engine='openpyxl', mode='a') as writer:
    potential_data_issue.to_excel(writer, sheet_name='Potential Data Issue', index=False)

In [233]:
email_body = "Hi Team, here is the Daily AR Offer Alert Report \n\n"
email_body += "Current Downtrending Threshold: 25% to 40%\n"
email_body += "Downtrending Swaps: Downtrend eCPM difference and Days Since Last Swap > 37\n"
email_body += "Underperforming Swaps: Recently swapped ARs with downtrend and Days Since Last Swap > 7 and <= 37\n"
email_body += "In Review - Swap: Swaps to be reviewed for the next 7 days\n"
email_body += "Below Cost eCPM: 7 Day eCPM is less than $7\n"
email_body += "Optout Rate Increase: Optout Rate % Diff >= 2% and Is Also Underperforming Swap\n\n"


email_body += "Please check the current downtrending ARs and swap:\n\n"
#add to email_body and for each DMA, add the count of downtrending swaps 
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {downtrending_swaps[downtrending_swaps['DMA'] == dma].shape[0]} downtrending swaps\n"

email_body += "\nPlease check the current underperforming swaps and swap back or to a new offer:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {underperforming_swaps[underperforming_swaps['DMA'] == dma].shape[0]} underperforming swaps\n"

email_body += "\nPlease check the recently swapped ARs that are in review and pay attention to performance changes:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {in_review[in_review['DMA'] == dma].shape[0]} swaps in review\n"

email_body += "\nPlease check the for ARs under cost eCPM and swap or remove the AR:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {below_cost[below_cost['DMA'] == dma].shape[0]} ARs below cost\n"

email_body += "\nPlease check the ARs with increased optout rates and swap:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {optout_rate_increase[optout_rate_increase['DMA'] == dma].shape[0]} swaps with an increased optout rate of 2%+\n"

email_body += "\nPlease check the ARs with potential data issues:\n\n"
for dma, df in dma_separated_dfs.items():
    if pd.notna(dma):
        email_body += f"{dma} - {potential_data_issue[potential_data_issue['DMA'] == dma].shape[0]} ARs with potential data issues\n"

In [234]:
#show me merged_df where shortcode_DP.SV is SVT_AL.PL.2
merged_df[merged_df['shortcode_DP.SV'] == 'SVT_AL.PL.2']

,shortcode_DP.SV,Ar Day,Offer ID,Revenue,Cost,Delivered,Jump Page Clicks,Clicks,Opportunity Cost,Optout,30 Day eCPM,30 Day CTR,30 Day JCTR,30 Day Optout Rate,7 Day eCPM,7 Day Optout Rate,eCPM % Diff,Optout Rate % Diff,Date Started,Days Since Swap,AR Check,DMA
113,SVT_AL.PL.2,0.0,12113,3175.13,220.16232,43683.0,3546.0,6246.0,2249.134625,1822.0,72.685713,14.298469,8.117574,4.170959,56.578823,4.549752,-22.159637,0.378793,2024-03-20,104.0,Good,Nathan Mai
114,SVT_AL.PL.2,1.0,12088,276.89,173.91276,34506.5,534.0,1849.5,-454.971384,910.5,8.024285,5.359860,1.547535,2.638633,6.477273,2.882687,-19.279132,0.244054,2024-06-27,5.0,In Review - Swap,Nathan Mai
115,SVT_AL.PL.2,2.0,11646,60.00,162.55008,32252.0,136.0,815.0,-625.881020,600.0,1.860350,2.526975,0.421679,1.860350,3.603459,2.049968,93.697950,0.189618,2023-10-04,272.0,Below Cost eCPM,Nathan Mai


In [235]:
# toaddr = ['sms@rxmg.com','offernotices@rxmg.com']
toaddr = ['nathan@rxmg.com']
today = datetime.today().strftime("%m_%d_%Y")
subject_line = f"SMS Daily AR Offer Alert Report - {today}"

for i in toaddr:
    send_email.send_email([filename],subject_line,email_body,i)